# Hyperparameter Optimization for 2-Arm ANCOVA NPE Model

This notebook performs **Bayesian Optimization** (Optuna) to find optimal neural network architecture for the 2-arm ANCOVA amortized posterior estimation model.

## Objectives
1. **Minimize calibration error** (mean absolute coverage error)
2. **Minimize parameter count** (model size)

This is a multi-objective optimization with Pareto-optimal solutions.

In [ ]:
import os

if not os.environ.get("KERAS_BACKEND"):
    os.environ["KERAS_BACKEND"] = "torch"
    
from pathlib import Path

import numpy as np
np.set_printoptions(suppress=True)
RNG = np.random.default_rng(2025)

import keras
import bayesflow as bf
import matplotlib.pyplot as plt

# Import ANCOVA-specific functions
from rctbp_bf_training.models.ancova.model import (
    ANCOVAConfig,
    create_ancova_adapter,
    create_simulator,
    create_validation_grid,
    get_model_metadata,
    save_model_with_metadata,
    create_ancova_objective,
    create_ancova_training_functions,
)

# Import infrastructure (for final model saving)
from rctbp_bf_training.core.infrastructure import params_dict_to_workflow_config

# Import Bayesian optimization infrastructure
from rctbp_bf_training.core import optimization
from rctbp_bf_training.core.optimization import (
    create_study,
    HyperparameterSpace,
    get_param_count,
    plot_optimization_results,
    summarize_best_trials,
    train_until_threshold,
    QualityThresholds,
)

# Create default configuration
config = ANCOVAConfig()
print(f"Config loaded: ancova_cont_2arms")
print(f"\nNetwork configs:")
print(f"  Summary network: {config.workflow.summary_network}")
print(f"  Inference network: {config.workflow.inference_network}")
print(f"\nOptuna available: {optimization.OPTUNA_AVAILABLE}")

In [ ]:
# Create simulator and adapter (prerequisites for optimization)
simulator = create_simulator(config, RNG)
adapter = create_ancova_adapter()

print("Setup complete:")
print(f"  Simulator ready with {len(simulator.sample(1).keys())} output keys")
print(f"  Adapter configured for ANCOVA model")

## Search Space and Optimization Grid

Define the hyperparameter search space and a reduced validation grid for faster optimization.

In [ ]:
# Define search space (customize ranges as needed)
search_space = HyperparameterSpace(
    # DeepSet
    summary_dim=(4, 16),
    deepset_width=(32, 128),
    deepset_depth=(1, 4),
    deepset_dropout=(0.05, 0.5),
    
    # CouplingFlow  
    flow_depth=(2, 8),
    flow_hidden=(32, 128),
    flow_dropout=(0.05, 0.5),
    
    # Training
    initial_lr=(1e-5, 5e-3),
    batch_size=(128, 832),
    
    # Fixed (not optimized)
    decay_rate=0.85,
    patience=10,
    window=20,
)

# Use factory function for validation grid
opt_conditions = create_validation_grid(extended=True)

print(f"Search space defined with {len(search_space.__dataclass_fields__)} parameters")
print(f"Optimization validation grid: {len(opt_conditions)} conditions")

In [ ]:
# Create objective function using factory
objective = create_ancova_objective(
    config=config,
    simulator=simulator,
    adapter=adapter,
    search_space=search_space,
    validation_conditions=opt_conditions,
    n_sims=500,
    n_post_draws=500,
    rng=RNG,
)

print("Objective function created via factory")
print(f"  Validation grid: {len(opt_conditions)} conditions")

## Create Study

Create the multi-objective Optuna study. Results are saved to SQLite for resumption.

In [5]:
# Create multi-objective study
study = create_study(
    study_name="ancova_cont_2arms",
    directions=["minimize", "minimize"],  # [calibration_error, param_count]
    storage="sqlite:///optuna_ancova_cont_2arms.db",  # Persistent storage for resumption
    load_if_exists=True,
)

print(f"Study created: {study.study_name}")
print(f"Existing trials: {len(study.trials)}")
print(f"Directions: {study.directions}")

[I 2026-01-11 15:58:16,780] A new study created in RDB with name: ancova_cont_2arms


Study created: ancova_cont_2arms
Existing trials: 0
Directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MINIMIZE: 1>]


In [6]:
# To monitor optimization in real-time, launch the Optuna Dashboard from a separate terminal:
#
#   python -m rctbp_bf_training.core.optimization --dashboard sqlite:///optuna_ancova_cont_2arms.db
#
# Or from Python:
#   from rctbp_bf_training.core.optimization import launch_dashboard
#   launch_dashboard("sqlite:///optuna_ancova_cont_2arms.db")
#
# Requires: pip install optuna-dashboard

print("Dashboard command:")
print("  python -m rctbp_bf_training.core.optimization --dashboard sqlite:///optuna_ancova_cont_2arms.db")

Dashboard command:
  python -m rctbp_bf_training.core.optimization --dashboard sqlite:///optuna_ancova_cont_2arms.db


## Run Optimization

Run the multi-objective optimization study.

In [7]:
# Run optimization (adjust n_trials based on compute budget)
# Each trial takes ~5-10 minutes depending on architecture
N_TRIALS = 100  # Adjust as needed

study.optimize(
    objective,
    n_trials=N_TRIALS,
    show_progress_bar=True,
    gc_after_trial=True,
)

print(f"\nOptimization complete!")
print(f"Total trials: {len(study.trials)}")
print(f"Pareto-optimal trials: {len(study.best_trials)}")

  0%|          | 0/100 [00:00<?, ?it/s]

INFO:bayesflow:Fitting on dataset instance of OnlineDataset.
INFO:bayesflow:Building on a test batch.


Epoch 1/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 364s 7s/step - loss: 3.2609 - val_loss: 1.0902 - moving_avg_val_loss: 1.0902
Epoch 2/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 285s 6s/step - loss: 3.5742 - val_loss: 1.2869 - moving_avg_val_loss: 1.1885
Epoch 3/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 304s 6s/step - loss: 9.4529 - val_loss: 2.4667 - moving_avg_val_loss: 1.6146
Epoch 4/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 284s 6s/step - loss: 1.2659 - val_loss: 0.3708 - moving_avg_val_loss: 1.3036
Epoch 5/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 284s 5s/step - loss: 6.8447 - val_loss: -0.3281 - moving_avg_val_loss: 0.9773
Epoch 6/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 211s 4s/step - loss: 0.6076 - val_loss: 0.9928 - moving_avg_val_loss: 0.9799
Epoch 7/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 279s 6s/step - loss: 2.2132 - val_loss: -0.8191 - moving_avg_val_loss: 0.7229
Epoch 8/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 341s 7s/step - loss: -0.4092 - val_loss: -0.7704 - moving_avg_val_loss: 0.5362
Epoch 9/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 310s 6s/step - loss: 2261.090

KeyboardInterrupt: 

## Analyze Results

Visualize the Pareto front and extract the best configurations.

In [ ]:
# Get best configurations from Pareto front
best_configs = summarize_best_trials(study)
display(best_configs)

# Print the best configuration for each objective
print("\n" + "="*60)
print("RECOMMENDED CONFIGURATIONS")
print("="*60)

if len(best_configs) > 0:
    # Best for calibration (lowest cal_error)
    best_cal = best_configs.iloc[0]
    print(f"\n📊 Best Calibration (trial {int(best_cal['trial'])}):")
    print(f"   Cal error: {best_cal['cal_error']:.4f}")
    print(f"   Params: {int(best_cal['param_count']):,}")
    
    # Best for size (if different)
    if "param_count" in best_configs.columns:
        best_size = best_configs.sort_values("param_count").iloc[0]
        if best_size["trial"] != best_cal["trial"]:
            print(f"\n📦 Smallest Model (trial {int(best_size['trial'])}):")
            print(f"   Cal error: {best_size['cal_error']:.4f}")
            print(f"   Params: {int(best_size['param_count']):,}")

In [ ]:
# Plot optimization results
from rctbp_bf_training.core.optimization import plot_optimization_results

fig = plot_optimization_results(study)
plt.show()

## Train Until Threshold (Optional)

Train the best configuration repeatedly until it meets the calibration error threshold.

In [ ]:
# Configuration for threshold-based training
thresholds = QualityThresholds(
    max_cal_error=0.05,
    max_c2st_deviation=0.05,
    max_coverage_error=0.05,
    max_iterations=40,
)

# Extended validation grid for final evaluation
final_conditions = create_validation_grid(extended=True)

print(f"Quality thresholds:")
print(f"  Max calibration error: {thresholds.max_cal_error}")
print(f"  Max C2ST deviation: {thresholds.max_c2st_deviation}")
print(f"  Max coverage error: {thresholds.max_coverage_error}")
print(f"  Max iterations: {thresholds.max_iterations}")
print(f"\nFinal validation grid: {len(final_conditions)} conditions")

In [ ]:
# Create training functions using factory
build_workflow_fn, train_fn, validate_fn = create_ancova_training_functions(
    simulator=simulator,
    adapter=adapter,
    validation_conditions=final_conditions,
    rng=RNG,
)

print("Training functions created via factory for train_until_threshold")

In [ ]:
# Get best hyperparameters from Pareto front and train until threshold
if len(best_configs) > 0:
    selected = best_configs.iloc[0]  # Best calibration
    trial_num = int(selected["trial"])
    trial = [t for t in study.trials if t.number == trial_num][0]
    best_params = trial.params
    
    print(f"Selected trial {trial_num} for threshold training")
    print(f"  Calibration error: {selected['cal_error']:.4f}")
    print(f"  Parameter count: {int(selected['param_count']):,}")
    
    # Train until threshold using the package function
    result = train_until_threshold(
        build_workflow_fn=build_workflow_fn,
        train_fn=train_fn,
        validate_fn=validate_fn,
        hyperparams=best_params,
        thresholds=thresholds,
        verbose=True,
    )
    
    if result["converged"]:
        print(f"\n{'='*60}")
        print("SUCCESS! Quality thresholds met")
        print(f"{'='*60}")
        print(f"Iterations: {result['iterations']}")
        print(f"Final scores:")
        for key, val in result["best_scores"].items():
            print(f"  {key}: {val:.4f}")
        
        best_workflow = result["workflow"]
    else:
        print(f"\nWarning: Did not meet thresholds after {result['iterations']} iterations")
        print(f"Best scores achieved:")
        for key, val in result["best_scores"].items():
            print(f"  {key}: {val:.4f}")
        
        best_workflow = result["workflow"]  # Use best attempt
else:
    print("No trials completed. Run optimization first.")
    best_workflow = None
    result = None

In [ ]:
# Save the best model with metadata
from pathlib import Path
from rctbp_bf_training.models.ancova.model import get_model_metadata, save_model_with_metadata

if best_workflow is not None and result is not None:
    # Get final workflow config from best_params
    final_workflow_config = params_dict_to_workflow_config(best_params)
    
    # Create ANCOVAConfig with optimized settings
    config_optimized = ANCOVAConfig(
        prior=config.prior,
        meta=config.meta,
        workflow=final_workflow_config,
    )
    
    # Save with metadata
    metadata = get_model_metadata(
        config=config_optimized,
        validation_results={
            "converged": result["converged"],
            "iterations": result["iterations"],
            "scores": result["best_scores"],
            "param_count": get_param_count(best_workflow.approximator),
        },
    )
    
    save_path = Path("checkpoints") / "ancova_cont_2arms_optimized"
    saved_path = save_model_with_metadata(best_workflow.approximator, save_path, metadata)
    
    print(f"\n✓ Model saved to: {saved_path}")
    print(f"✓ Metadata saved to: {saved_path.with_suffix('.json')}")
else:
    print("No model to save. Run optimization and threshold training first.")